<a href="https://colab.research.google.com/github/jeniferGoncalvesDaSilvaDev/maya_chat_bot/blob/main/Maya_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install nltk
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 63.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 55.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
pip install streamlit transformers nltk sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 51.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 63.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.3 MB/s eta 0:00:00


In [ ]:
import streamlit as st
from transformers import pipeline
import random
import nltk
from sentence_transformers import SentenceTransformer, util

# Baixar pacotes necessários para tokenização NLTK
nltk.download('punkt')

# Carregar o modelo de análise de sentimentos
sentiment_analyzer = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment")

# Carregar o modelo de embeddings para análise semântica
modelo_embeddings = SentenceTransformer('all-MiniLM-L6-v2')

# Histórico de interações
historico_interacao = []

# Respostas baseadas no sentimento
respostas_positivas = [
    "Fico muito feliz em saber disso! Quer me contar mais?",
    "Isso é ótimo! O que mais te animou hoje?",
    "Adoro ouvir boas notícias! O que mais você gostaria de compartilhar?"
]

respostas_negativas = [
    "Sinto muito que esteja passando por isso. Quer conversar mais sobre o que está te incomodando?",
    "Lamento saber disso. Posso te ajudar de alguma forma?",
    "Estou aqui para ouvir. Me conta mais sobre o que está acontecendo."
]

respostas_neutras = [
    "Entendo. Pode me explicar um pouco mais?",
    "Interessante! Como você se sente em relação a isso?",
    "Isso faz sentido. Pode me dar mais detalhes?"
]

# Função para analisar o sentimento e responder
def analisar_sentimento_responder(texto_usuario):
    analise = sentiment_analyzer(texto_usuario)[0]
    sentimento = analise['label']
    score = analise['score']

    if sentimento == "LABEL_2":
        sentimento = "POSITIVE"
    elif sentimento == "LABEL_1":
        sentimento = "NEUTRAL"
    else:
        sentimento = "NEGATIVE"

    # Adicionar ao histórico de interações
    historico_interacao.append({"texto": texto_usuario, "sentimento": sentimento, "score": score})

    # Escolher resposta apropriada
    if sentimento == "POSITIVE":
        resposta = random.choice(respostas_positivas)
    elif sentimento == "NEGATIVE":
        resposta = random.choice(respostas_negativas)
    else:
        resposta = random.choice(respostas_neutras)

    return resposta, sentimento, score

# Função para encontrar respostas semelhantes na memória
def encontrar_resposta_semelhante(nova_entrada):
    if not historico_interacao:
        return None

    # Criar embedding da nova entrada
    embedding_nova_entrada = modelo_embeddings.encode(nova_entrada, convert_to_tensor=True)

    # Criar embeddings para interações anteriores
    embeddings_historico = [modelo_embeddings.encode(entry["texto"], convert_to_tensor=True) for entry in historico_interacao]

    # Calcular similaridade
    similaridades = [util.pytorch_cos_sim(embedding_nova_entrada, emb)[0][0].item() for emb in embeddings_historico]

    # Encontrar a interação mais semelhante
    indice_mais_similar = similaridades.index(max(similaridades))

    if similaridades[indice_mais_similar] > 0.7:
        return historico_interacao[indice_mais_similar]["texto"]

    return None

# Título da aplicação Streamlit
st.title("maya-bot")

# Entrada do usuário
texto_usuario = st.text_input("Como você está se sentindo hoje?")

if texto_usuario:
    # Verifica se já houve uma conversa semelhante antes
    resposta_semelhante = encontrar_resposta_semelhante(texto_usuario)
    if resposta_semelhante:
        st.write(f"Você já mencionou algo parecido antes: '{resposta_semelhante}'. Quer me contar mais sobre isso?")
    else:
        resposta, sentimento, score = analisar_sentimento_responder(texto_usuario)
        st.write(f"Mayá ({sentimento}, confiança: {score:.2f}): {resposta}")

    # Mostrar o histórico de interações
    #if len(historico_interacao) > 0:
        #st.subheader("Histórico de Interações:")
        #for entry in historico_interacao:
            #st.write(f"Texto: {entry['texto']} - Sentimento: {entry['sentimento']} (Confiança: {entry['score']:.2f})")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Device set to use cpu
2025-03-13 21:21:09.449 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-13 21:21:09.621 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-03-13 21:21:09.622 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-13 21:21:09.625 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-13 21:21:09.627 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-13 21:21:09.628 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-13 21:21:09.628 Thread 'MainThre